<a href="https://colab.research.google.com/github/vishallmaurya/sentiment_analyzer_backend/blob/main/incremental_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 7.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sentence_transformers import SentenceTransformer
import emoji
import numpy as np
from sklearn.metrics import accuracy_score
from text_preprocessing import cleaning_tweets
from sklearn.linear_model import SGDClassifier
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
incremental_clf = pickle.load(open("/content/model.pkl", "rb"))
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
data = pd.read_csv("/content/synthetic_sentiment_data.csv")

In [ ]:
data.head(1)

,Tweet,gpt_label
0,I had the worst experience with their customer...,0


In [ ]:
data.drop(['Unnamed: 0', 'Date', 'class'], inplace=True, axis=1)

In [ ]:
cleaned_tweets = []
emoji_list = []

for i in range(data.shape[0]):
  clean_text, extracted_emojis = cleaning_tweets(str(data.iloc[i]["Tweet"]))
  cleaned_tweets.append(clean_text)
  emoji_list.append(extracted_emojis)

data["clean_tweet"] = cleaned_tweets
data["extracted_emojis"] = emoji_list

In [ ]:
def emoji_to_text(emoji_string):
    return ' '.join(emoji.demojize(e) for e in emoji_string) if emoji_string else ''

data["emoji_text"] = data["extracted_emojis"].apply(emoji_to_text)

In [ ]:
text_embeddings = model.encode(data["clean_tweet"].tolist(), convert_to_numpy=True)
embedding_dim = model.get_sentence_embedding_dimension()
emoji_embeddings = [
    model.encode(emoji_text, convert_to_numpy=True) if emoji_text.strip() else np.zeros(embedding_dim)
    for emoji_text in data["emoji_text"]
]

In [ ]:
X_new = np.hstack((text_embeddings, np.array(emoji_embeddings)))

y_new = data["gpt_label"].values

In [ ]:
class_weights = compute_class_weight("balanced", classes=np.unique(y_new), y=y_new)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

In [ ]:
class_weight_dict

{0: 1.0, 1: 1.0, 2: 1.0}

In [ ]:
incremental_clf.class_weight = class_weight_dict
incremental_clf.partial_fit(X_new, y_new, classes=np.array([0, 1, 2]))

SGDClassifier(class_weight={0: 1.0, 1: 1.0, 2: 1.0}, loss='perceptron',
              warm_start=True)

In [ ]:
y_pred_new = incremental_clf.predict(X_new)
accuracy = accuracy_score(y_new, y_pred_new)
print(f"Incremental Learning Accuracy: {accuracy:.4f}")

Incremental Learning Accuracy: 1.0000


In [21]:
pickle.dump(incremental_clf, open("model.pkl", "wb"))